# Rodar todos os códigos do diretório

In [47]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

def run_notebook(notebook_path, output_path=None):
    """
    Executa um notebook e opcionalmente salva o resultado no mesmo formato do notebook.
    """
    # Abrir o notebook
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Configurar o executor
    executor = ExecutePreprocessor(timeout=600, kernel_name='python3')
    
    # Executar o notebook
    executor.preprocess(notebook, {'metadata': {'path': './'}})
    
    # Salvar o notebook executado, se output_path for fornecido
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            nbformat.write(notebook, f)

def save_notebook_outputs_to_txt(notebook_path, output_txt_path):
    """
    Lê um notebook executado e salva as saídas das células no formato TXT.
    """
    # Ler o notebook executado
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook_content = nbformat.read(f, as_version=4)
    
    # Salvar as saídas no arquivo TXT
    with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
        for cell in notebook_content['cells']:
            if cell['cell_type'] == 'code' and 'outputs' in cell:
                txt_file.write("\n# Saída\n")
                for output in cell['outputs']:
                    if 'text' in output:
                        txt_file.write(output['text'] + '\n')
                    elif 'data' in output and 'text/plain' in output['data']:
                        txt_file.write(output['data']['text/plain'] + '\n')

# Caminhos dos notebooks e saída
abordagem_notebook = './abordagem.ipynb'
processamento_notebook = './processamento.ipynb'
processamento_output_txt = './processamento_executado.txt'

# Executar o primeiro notebook (abordagem.ipynb)
print("Executando abordagem.ipynb...")
run_notebook(abordagem_notebook)

# Executar o segundo notebook (processamento.ipynb)
print("Executando processamento.ipynb...")
run_notebook(processamento_notebook)

# Salvar as saídas do notebook "processamento.ipynb" no arquivo TXT
print("Salvando saídas do processamento.ipynb...")
save_notebook_outputs_to_txt(processamento_notebook, processamento_output_txt)

print("Processamento concluído. Saída salva em:", processamento_output_txt)


Executando abordagem.ipynb...
Executando processamento.ipynb...


2024-12-05 10:54:38.416425: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 10:54:38.419983: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-05 10:54:38.462190: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:54:39.478289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Salvando saídas do processamento.ipynb...
Processamento concluído. Saída salva em: ./processamento_executado.txt


In [49]:
import pandas as pd
import re

# Caminho do arquivo
processamento_output_txt = './processamento_executado.txt'

# Inicializar lista para armazenar os resultados
results = []

# Processar o arquivo
with open(processamento_output_txt, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        
        # Detectar nome do dataset e modelo
        if line.startswith("Arquivo:"):
            parts = line.split(" - ")
            current_dataset = parts[0].split(":")[1].strip()
            current_model = parts[1] if len(parts) > 1 else None
            
            # Redefinir métricas para o novo bloco
            mse = None
            rmse = None
            mae = None
            mape = None
            r2 = None
            explained_var = None
            max_err = None

        # Detectar métricas
        elif "Mean Squared Error" in line:
            mse = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Root Mean Squared Error" in line:
            rmse = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Mean Absolute Error" in line:
            mae = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Mean Absolute Percentage Error" in line:
            mape = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "R² Score" in line:
            r2 = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Explained Variance Score" in line:
            explained_var = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        elif "Max Error" in line:
            max_err = float(re.search(r"[-+]?\d*\.\d+|\d+", line).group())
        
        # Adicionar resultados ao final do bloco
        elif line.startswith("--------------------------------------------------"):
            results.append({
                "Dataset": current_dataset,
                "Model": current_model,
                "MSE": mse,
                "RMSE": rmse,
                "MAE": mae,
                "MAPE": mape,
                "R²": r2,
                "Explained Variance": explained_var,
                "Max Error": max_err,
            })

# Criar DataFrame
df_results = pd.DataFrame(results)

# Salvar os resultados em um arquivo CSV
df_results.to_csv('./resultados/4/tabela_resultados_processamento.csv', index=False)

# Exibir os primeiros resultados diretamente no terminal
print(df_results.head(10))


                  Dataset             Model       MSE  RMSE       MAE  \
0    cleaned_cocomo81.txt  Regressão Linear  0.186184  None  0.083779   
1  cleaned_desharnais.txt  Regressão Linear  0.175406  None  0.120379   
2       cleaned_isbsg.txt  Regressão Linear  0.108375  None  0.100613   
3     cleaned_maxwell.txt  Regressão Linear  0.147751  None  0.098961   
4       cleaned_china.txt  Regressão Linear  0.019632  None  0.007178   
5    cleaned_cocomo81.txt   Regressão Ridge  0.060793  None  0.048836   
6  cleaned_desharnais.txt   Regressão Ridge  0.136007  None  0.087269   
7       cleaned_isbsg.txt   Regressão Ridge  0.118609  None  0.097567   
8     cleaned_maxwell.txt   Regressão Ridge  0.041876  None  0.034528   
9       cleaned_china.txt   Regressão Ridge  0.028778  None  0.012308   

           MAPE        R²  Explained Variance  Max Error  
0  5.251383e+02  0.295982            0.377891   0.776781  
1  7.977373e-01 -0.389023           -0.383452   0.491923  
2  2.021612e+00  0.